In [16]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/fukami/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
def tokenizer(text):
    return text.split()

In [21]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes ruuning and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'ruun', 'run', 'lot']

In [3]:
df = pd.read_csv('movie_data.csv')

In [29]:
df.head(3)

,review,sentiment
0,I read a few reviews of the movie and got the ...,1
1,I hate to comment on something I didn't finish...,0
2,Only on a very rare occasion does an episode o...,0


In [6]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_train = df.loc[25000:, 'review'].values
y_train = df.loc[25000:, 'sentiment'].values

In [11]:
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)
param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
              'vect__stop_words': [stop, None],
              'vect__tokenizer': [tokenizer, tokenizer_porter],
              'vect__use_idf': [False],
              'vect__norm':[None],
              'clf__penalty': ['l1', 'l2'],
              'clf__C': [1.0, 10.0, 100.0]}
             ]

In [25]:
lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf', LogisticRegression(random_state=0))])

In [33]:
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                          scoring='accuracy',
                          cv=5, verbose=1,
                          n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 43.5min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 56.9min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__tokenizer': [<function tokenizer at 0x7f6f2aab0bf8>, <function tokenizer_porter at 0x7f6f2aab0950>], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'h...asn', 'weren', 'won', 'wouldn'], None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=1

In [36]:
print('Best parameter set: %s' % gs_lr_tfidf.best_params_)

Best parameter set: {'vect__ngram_range': (1, 1), 'clf__C': 10.0, 'vect__tokenizer': <function tokenizer at 0x7f6f2aab0bf8>, 'vect__stop_words': None, 'clf__penalty': 'l2'}


In [39]:
import pickle
import os

In [40]:
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

In [41]:
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
           protocol=4)

In [42]:
pickle.dump(clf,
           open(os.path.join(dest, 'classifier.pkl'), 'wb'),
           protocol=4)

NameError: name 'clf' is not defined